In [38]:
import warnings
warnings.filterwarnings('ignore')


In [39]:

import numpy as np
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import os
from glob import glob

In [40]:
IMAGE_SIZE = [224, 224]
TRAIN_PATH = 'Datasets/train'
VALID_PATH = 'Datasets/test'
BATCH_SIZE = 32
EPOCHS = 10
MODEL_PATH = 'chest_xray.h5'

In [41]:
def create_model():
    base_model = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)
    for layer in base_model.layers:
        layer.trainable = False

    x = Flatten()(base_model.output)
    output_layer = Dense(len(glob(f'{TRAIN_PATH}/*')), activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=output_layer)
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [42]:
def get_data_generators():
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True
    )
    
    test_datagen = ImageDataGenerator(rescale=1./255)
    
    train_generator = train_datagen.flow_from_directory(
        TRAIN_PATH,
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical'
    )
    
    validation_generator = test_datagen.flow_from_directory(
        VALID_PATH,
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical'
    )
    
    return train_generator, validation_generator


In [43]:
def train_model(model, train_generator, validation_generator):
    callbacks = [
        EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
        ModelCheckpoint(MODEL_PATH, monitor='val_loss', save_best_only=True)
    ]
    
    history = model.fit(
        train_generator,
        validation_data=validation_generator,
        steps_per_epoch=len(train_generator),
        validation_steps=len(validation_generator),
        callbacks=callbacks,
        epochs=EPOCHS,

    )
    
    return history

In [44]:
def predict_image(model, img_path):
    img = load_img(img_path, target_size=IMAGE_SIZE)
    x = img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    prediction = model.predict(x)
    return prediction

In [45]:
model = create_model()
train_gen, val_gen = get_data_generators()
history = train_model(model, train_gen, val_gen)
model = load_model(MODEL_PATH)
img_path = 'Datasets/val/NORMAL/NORMAL2-IM-1431-0001.jpeg'
prediction = predict_image(model, img_path)
result = np.argmax(prediction)
if result == 0:
    print("Person is Affected By PNEUMONIA")
else:
    print("Result is Normal")

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Epoch 1/10
  3/163 [..............................] - ETA: 8:20 - loss: 0.6648 - accuracy: 0.7292

KeyboardInterrupt: 

In [46]:
!streamlit run streamlit_app.py


  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8502
  Network URL: http://192.168.200.140:8502

1/1 [==============================] - 0s 158ms/step
^C
  Stopping...
